In [2]:
import pandas as pd

In [7]:
labels_description = {}

df = pd.read_csv("./(V1.7)all_products_data_set.csv")
for row in range(df.shape[0]):
    label_2 = df.loc[row]["label_2nd"]
    production_description = df.loc[row]["raw_text"]
    if pd.isna(label_2) or pd.isna(production_description):
        continue
    if label_2 not in labels_description.keys():
        labels_description.update({label_2:set()})
    if not pd.isna(production_description) and production_description.__contains__("match"):
        
        labels_description.get(label_2).add(production_description)


In [10]:
count_known = 0
count_known_keys = []
for key in labels_description.keys():
    if len(labels_description.get(key)) > 0:
        print(key, len(labels_description.get(key)))
        count_known += 1
        count_known_keys.append(key)
print(count_known)  

Swimwear 89
Jewelry 9
Others 8
Homeware 13
Headwear 10
Pants/Trousers 18
Bag 11
Dresses 6
Lingerie 6
Short 14
Coats/Jackets 5
Skirts 3
Belt 1
Sandal 1
Glasses 11
Skin Care 5
Sweaters 9
Shirts 27
Scarf 6
Sock 7
Tank Tops 7
Leggings 2
Activewear 5
Glove 1
Scrunchie 7
Wallet 8
Watch 2
Skateboard 9
28


In [3]:
df_samples = pd.read_csv("/Users/luis/Downloads/products-June-28th.csv", nrows=100)

In [3]:
df_samples.to_csv("test_input.csv", index=False)

NameError: name 'df_samples' is not defined

In [22]:
df = pd.read_csv("../tools/(V1.7)all_products_data_set.csv")

In [23]:
df.columns

Index(['index', 'id', 'title', 'tags', 'images', 'gender', 'product_type',
       'product_type(modified)', 'main_category', 'sub_category',
       'match_most_similar_>80%_string', 'match_most_similar_>60%_string',
       'label_1st', 'label_2nd', 'label_3rd', 'buckets_num', 'pattern_info',
       'buckets', 'color_info', 'color_num', 'colors', 'url', 'raw_text'],
      dtype='object')

In [33]:
colorname_to_hex = {}
for row in range(df.shape[0]):
    color_info = df.loc[row]["color_info"]
    if color_info == "" or pd.isna(color_info):
        continue
    color_info_list = color_info.split(",")
    for item in color_info_list:
        c2h = item.split("(")
        color_name = c2h[0].lower().strip()
        hex = c2h[1].removesuffix(")")
        if color_name not in colorname_to_hex.keys():
            colorname_to_hex.update({color_name:hex})

    

In [37]:
import json
with open("json_files/colorsname_to_hex.json", "w") as f1:
    json.dump(colorname_to_hex, f1,indent=6)

In [21]:
import requests
from bs4 import BeautifulSoup
def requery_func(colors_str):
    res = set()
    pattern = set()
    if colors_str == "" or isinstance(colors_str, float):
        return "", ""
    colors_list = colors_str.split(",")
    #print(row, colors_list)
    for item in colors_list:
        URL = "https://colornames.org/search/results/?type=exact&query="
        URL += item
        try:
            page = requests.get(URL)
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find_all("a", class_="button is-fullwidth freshButton")
            #the first result which has the highest vote
            content = results[0].find("span").text.strip()
            res.add(content)
        except:
            pattern.add(item)
    return ",".join(res), ",".join(pattern)


In [40]:
item = "black pink"
colorname= " ".join([e.capitalize() for e in item.split(" ")])
colorname

'Black Pink'